In [1]:
import glob
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
from scipy import ndimage
import pickle
import nibabel as nib
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision



In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list

def find_centroid(bv_img):
    bv_voxel_num = np.sum(bv_img)
    # find x_centroid
    x_centroid = 0
    for i in range(bv_img.shape[0]):
        x_centroid += np.sum(bv_img[i,:,:])*i
    x_centroid /= bv_voxel_num
    # find y_centroid
    y_centroid = 0
    for i in range(bv_img.shape[1]):
        y_centroid += np.sum(bv_img[:,i,:])*i
    y_centroid /= bv_voxel_num
    # find z_centroid
    z_centroid = 0
    for i in range(bv_img.shape[2]):
        z_centroid += np.sum(bv_img[:,:,i])*i
    z_centroid /= bv_voxel_num
    return (round(x_centroid), round(y_centroid), round(z_centroid))


In [3]:
def even_func(x):
    if x % 2 == 0:
        return x
    else:
        return x+1
    
def padded_minimum_size(label, min_size):
    img_size= np.shape(label)
    x_offset=0
    y_offset=0
    z_offset=0
    x_flag=False
    y_flag=False
    z_flag=False
    if img_size[0]<min_size:
        x_offset=min_size-img_size[0]
        x_flag=True
    if img_size[1]<min_size:
        y_offset=min_size-img_size[1]
        y_flag=True
    if img_size[2]<min_size:
        z_offset=min_size-img_size[2]
        z_flag=True
    
    if x_flag == True and y_flag == True and z_flag == True:
        padded_label=np.zeros((min_size,min_size,min_size),np.uint8)
        
        padded_label[round(x_offset/2):round(x_offset/2)+img_size[0], 
                     round(y_offset/2):round(y_offset/2)+img_size[1], 
                     round(z_offset/2):round(z_offset/2)+img_size[2]]=label
        
        return padded_label
    
    elif x_flag == True and y_flag == True:
        padded_label=np.zeros((min_size,min_size,even_func(img_size[2])),np.uint8)
        
        padded_label[round(x_offset/2):round(x_offset/2)+img_size[0],
                     round(y_offset/2):round(y_offset/2)+img_size[1],
                     0:img_size[2]] = label
        return padded_label
    
    elif x_flag == True and z_flag == True:
        padded_label=np.zeros((min_size,even_func(img_size[1]),min_size),np.uint8)
        
        padded_label[round(x_offset/2):round(x_offset/2)+img_size[0],
                     0:img_size[1],
                     round(z_offset/2):round(z_offset/2)+img_size[2]]=label
        return padded_label
    
    elif y_flag == True and z_flag == True:
        padded_label=np.zeros((even_func(img_size[0]),min_size,min_size),np.uint8)
        
        padded_label[0:img_size[0],
                     round(y_offset/2):round(y_offset/2)+img_size[1],
                     round(z_offset/2):round(z_offset/2)+img_size[2]]=label
        return padded_label
    
    elif x_flag == True:
        padded_label=np.zeros((min_size,even_fuc(img_size[1]),even_func(img_size[2])),np.uint8)
      
        padded_label[round(x_offset/2):round(x_offset/2)+img_size[0],
                     0:img_size[1],
                     0:img_size[2]]=label
        return padded_label
    
    elif y_flag == True:
        padded_label=np.zeros((even_func(img_size[0]),min_size,even_func(img_size[2])),np.uint8)
       
        padded_label[0:img_size[0],
                     round(y_offset/2):round(y_offset/2)+img_size[1],
                     0:img_size[2]]=label
        return padded_label
    
    elif z_flag == True:
        padded_label=np.zeros((even_func(img_size[0]),even_func(img_size[1]),min_size),np.uint8)

        padded_label[0:img_size[0],
                     0:img_size[1],
                     round(z_offset/2):round(z_offset/2)+img_size[2]]=label
        return padded_label
    
    else:
        padded_label=np.zeros((even_func(img_size[0]),even_func(img_size[1]),even_func(img_size[2])),np.uint8)
       
        padded_label[0:img_size[0],
                     0:img_size[1],
                     0:img_size[2]]=label
        return padded_label

def extract_positive(whole_label,box_size,smallest_ratio,step_size,mul_label,img_idx):
    img_size=np.shape(whole_label)
    bv_voxel_num=np.sum(whole_label)
    positive_sub_volumes=[]
    x_slice,y_slice,z_slice = ndimage.find_objects(whole_label)[0]
    offset=5
    
    x_start = x_slice.stop-offset if x_slice.stop-offset >= box_size else box_size
    x_stop = x_slice.start+box_size+offset if x_slice.start+box_size+offset <= img_size[0] else img_size[0]
    
    y_start = y_slice.stop-offset if y_slice.stop-offset >= box_size else box_size
    y_stop = y_slice.start+box_size+offset if y_slice.start+box_size+offset <= img_size[1] else img_size[1]
    
    z_start = z_slice.stop-offset if z_slice.stop-offset >= box_size else box_size
    z_stop = z_slice.start+box_size+offset if z_slice.start+box_size+offset <= img_size[2] else img_size[2]
    
    for i in range(x_start,x_stop+1,step_size):
        for j in range(y_start,y_stop+1,step_size):
            for k in range(z_start,z_stop+1,step_size):
                contain_ratio = np.sum(whole_label[i-box_size:i,
                                                   j-box_size:j,
                                                   k-box_size:k])/(bv_voxel_num+0.001)
            
                if contain_ratio > smallest_ratio:
                    positive_sub_volumes.append((img_idx,mul_label,(i,j,k)))
    return positive_sub_volumes



In [4]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
data_base_path = '/scratch/zq415/grammar_cor/Localization/data'
data_folder_list = ['20180419_newdata_nii_with_filtered', 'new_data_20180522_nii', 'organized_data_nii']

In [5]:
all_BVs = []
for cur_floder in data_folder_list:
    cur_folder_path = os.path.join(data_base_path,cur_floder)
    all_BVs += glob.glob(cur_folder_path+'/*/*/*[Bb][Vv]*')
print(len(all_BVs))

574


In [6]:
all_data_dic = {}
same_name_num = 0
for full_bv_path in all_BVs:
    bv_file_name = os.path.basename(full_bv_path)
    if 'BV' in bv_file_name:
        if bv_file_name[:-14] in all_data_dic:
            same_name_num += 1
            continue
        all_data_dic[bv_file_name[:-14]] = full_bv_path
    else:
        if bv_file_name[:-9] in all_data_dic:
            same_name_num += 1
            continue
        all_data_dic[bv_file_name[:-9]] = full_bv_path

In [ ]:
# all_data_dic = {}
# same_name_num = 0
# for full_bv_path in all_BVs:
#     bv_file_name = os.path.basename(full_bv_path)
#     if 'BV' in bv_file_name:
#         if bv_file_name[:-14] in all_data_dic:
#             same_name_num += 1
#             continue
#         all_data_dic[bv_file_name[:-14]] = (full_bv_path[:-14] + '.nii', full_bv_path)
#     else:
#         if bv_file_name[:-9] in all_data_dic:
#             same_name_num += 1
#             continue
#         all_data_dic[bv_file_name[:-9]] = (full_bv_path[:-9] + '_2.nii', full_bv_path)



In [ ]:
# num = 0 

# for key in all_data_dic:
#     #print(all_data_dic[key])
# #     print(key)
#     if 'new_data_20180522_nii' in all_data_dic[key]:
#         print(num)
#     num += 1 

In [ ]:
# from shutil import copyfile
# dst = '/scratch/zq415/grammar_cor/mutant_detect/data/mutant_img/'

# all_train_data = []
# all_idx = []
# img_num = 0
# normal_num = 0
# mutant_num = 0
# for key in all_data_dic:
#     if key in mutant_names:
#         bv_base_name = os.path.basename(all_data_dic[key][1])
#         copyfile(all_data_dic[key][1], dst + bv_base_name)
        
#         img_base_name = os.path.basename(all_data_dic[key][0])
#         copyfile(all_data_dic[key][0], dst + img_base_name)
        
# #         bv_label = nib.load(all_data_dic[key][1])
# #         bv_label = bv_label.get_data()
# #         #bv_label = padded_minimum_size(bv_label.get_data(),128)

# #         img = nib.load(all_data_dic[key][0])
# #         img = img.get_data()
# #         #img = padded_minimum_size(img.get_data(),128)
        
        


In [ ]:
# all_train_data = []
# all_idx = []
# img_num = 0
# normal_num = 0
# mutant_num = 0
# for key in all_data_dic:
#     bv_label = nib.load(all_data_dic[key])
#     bv_label = padded_minimum_size(bv_label.get_data(),128)
#     bv_label=np.round(zoom(bv_label,1/2))
#     #bv_centroid = find_centroid(bv_label)
#     all_train_data.append(bv_label)
#     if key in mutant_names:
# #def extract_positive(whole_label,box_size,smallest_ratio,step_size,mul_label,img_idx):
#         all_idx += extract_positive(bv_label,64,0.94,2,0,img_num)
#         mutant_num += 1
#         img_num += 1
#     else:
#         all_idx += extract_positive(bv_label,64,0.99,3,1,img_num)
#         normal_num += 1
#         img_num += 1
#     print('train sample number: ', img_num)
#     assert(len(all_train_data)==img_num)
# print('mutant number: ', mutant_num)
# print('normal number: ', normal_num)
# print('train sample number: ', len(all_idx))

In [ ]:
#all_data = [all_train_data, all_idx]
#save_name = 'All_train_bv_data.pickle'
#save_file = open(os.path.join(os.getcwd(),'data',save_name),'wb')
#pickle.dump(all_data,save_file)
#save_file.close()

In [7]:
save_name = 'All_train_bv_data.pickle'
with open(os.path.join(os.getcwd(),'data',save_name), "rb") as input_file:
    all_train_data, all_idx = pickle.load(input_file)

In [8]:
mut_sample_num = 0
normal_sample_num = 0
for i in range(len(all_idx)):
    if all_idx[i][1] == 0:
        mut_sample_num += 1
    if all_idx[i][1] == 1:
        normal_sample_num += 1
print(mut_sample_num, normal_sample_num, len(all_idx))

147983 131853 279836


In [9]:
mutant_list = []
normal_list = []
img_num = 0
for i in range(len(all_idx)):
    if all_idx[i][1] == 0:
        mutant_list.append(all_idx[i][0])
    else:
        normal_list.append(all_idx[i][0])
print(len(set(mutant_list)), list(np.sort(list(set(mutant_list)))))
print(len(set(normal_list)), list(np.sort(list(set(normal_list)))))

103 [1, 3, 25, 48, 49, 51, 84, 91, 92, 93, 95, 103, 105, 124, 135, 139, 152, 153, 186, 187, 188, 189, 190, 192, 193, 194, 206, 216, 218, 221, 234, 235, 237, 239, 253, 255, 256, 257, 258, 268, 290, 293, 294, 300, 304, 305, 306, 307, 312, 316, 333, 334, 336, 340, 341, 352, 356, 357, 358, 359, 360, 361, 362, 363, 365, 366, 405, 407, 413, 416, 417, 419, 420, 421, 427, 428, 429, 434, 438, 440, 451, 452, 453, 454, 455, 458, 459, 460, 484, 496, 497, 499, 507, 508, 509, 520, 521, 523, 546, 548, 553, 555, 557]
462 [0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 94, 96, 97, 98, 99, 100, 101, 102, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 

In [10]:
mutant_list = list(np.sort(list(set(mutant_list))))
normal_list = list(np.sort(list(set(normal_list))))
print('mutant: ', mutant_list)
print('normal: ', normal_list)

mutant:  [1, 3, 25, 48, 49, 51, 84, 91, 92, 93, 95, 103, 105, 124, 135, 139, 152, 153, 186, 187, 188, 189, 190, 192, 193, 194, 206, 216, 218, 221, 234, 235, 237, 239, 253, 255, 256, 257, 258, 268, 290, 293, 294, 300, 304, 305, 306, 307, 312, 316, 333, 334, 336, 340, 341, 352, 356, 357, 358, 359, 360, 361, 362, 363, 365, 366, 405, 407, 413, 416, 417, 419, 420, 421, 427, 428, 429, 434, 438, 440, 451, 452, 453, 454, 455, 458, 459, 460, 484, 496, 497, 499, 507, 508, 509, 520, 521, 523, 546, 548, 553, 555, 557]
normal:  [0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 94, 96, 97, 98, 99, 100, 101, 102, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 

In [11]:
#img_nft = nib.Nifti1Image(np.squeeze(all_train_data[193]),np.eye(4))
#img_save_data_path = './img/mul_img.nii'
#nib.save(img_nft,img_save_data_path)

In [29]:
print(len(test_mutant_list))
print(len(test_normal_list))

15
84


In [28]:
test_mutant_list = [i for i in mutant_list if i in range(255,320)]
test_normal_list = [i for i in normal_list if i in range(255,370)]
train_mutant_list = [x for x in mutant_list if x not in test_mutant_list]
train_normal_list = [x for x in normal_list if x not in test_normal_list]
print('test_mutant_list', test_mutant_list)
print('train_mutant_list', train_mutant_list)
print('test_normal_list', test_normal_list)
print('train_normal_list', train_normal_list)
test_img_list = test_mutant_list + test_normal_list
train_img_list = train_mutant_list + train_normal_list

test_mutant_list [255, 256, 257, 258, 268, 290, 293, 294, 300, 304, 305, 306, 307, 312, 316]
train_mutant_list [1, 3, 25, 48, 49, 51, 84, 91, 92, 93, 95, 103, 105, 124, 135, 139, 152, 153, 186, 187, 188, 189, 190, 192, 193, 194, 206, 216, 218, 221, 234, 235, 237, 239, 253, 333, 334, 336, 340, 341, 352, 356, 357, 358, 359, 360, 361, 362, 363, 365, 366, 405, 407, 413, 416, 417, 419, 420, 421, 427, 428, 429, 434, 438, 440, 451, 452, 453, 454, 455, 458, 459, 460, 484, 496, 497, 499, 507, 508, 509, 520, 521, 523, 546, 548, 553, 555, 557]
test_normal_list [259, 260, 261, 262, 263, 264, 265, 266, 267, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 292, 295, 296, 297, 298, 299, 301, 302, 303, 308, 309, 310, 311, 313, 314, 315, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 335, 337, 338, 339, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 364, 367, 368, 369]
train_normal_list [0, 2

In [ ]:
# test_mutant_list = mutant_list[2:len(mutant_list):9]
# test_normal_list = normal_list[2:len(normal_list):9]
# train_mutant_list = [x for x in mutant_list if x not in test_mutant_list]
# train_normal_list = [x for x in normal_list if x not in test_normal_list]
# print('test_mutant_list', test_mutant_list)
# print('train_mutant_list', train_mutant_list)
# print('test_normal_list', test_normal_list)
# print('train_normal_list', train_normal_list)
# test_img_list = test_mutant_list + test_normal_list
# train_img_list = train_mutant_list + train_normal_list

In [30]:
test_idx = []
train_idx = []
for i in range(len(all_idx)):
    if all_idx[i][0] in test_img_list:
        test_idx.append(all_idx[i])
    elif all_idx[i][0] in train_img_list:
        train_idx.append(all_idx[i])
    else:
        print('something wrong')

In [31]:
from torch.utils.data import Dataset, DataLoader

class Mouse_sub_volumes(Dataset):
    """Mouse sub-volumes BV dataset."""

    def __init__(self, all_whole_volumes, all_idx, transform=None):
        """
        Args:
            all_whole_volumes: Contain all the padded whole BV volumes as a dic
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.whole_volumes = all_whole_volumes
        self.idx = all_idx
        self.transform = transform
    def __len__(self):
        return len(self.idx)

    def __getitem__(self, num):
        #idx [0] dictionary index, [1] label(0 or 1), [2] x, y, z sub-volumes index
        box_size=64
        current_img = self.idx[num][0]
        label = self.idx[num][1]
        x, y, z = self.idx[num][2]
        img = self.whole_volumes[current_img][x-box_size:x,
                                             y-box_size:y,
                                             z-box_size:z]
        img = np.float32(img[np.newaxis,...])
        sample = {'image': img, 'label': label, 'img_idx': current_img}
        return sample

In [32]:
class Rotate(object):
    
    """
    Rotate the image for data augmentation, but prefer original image.
    """
    
    def __init__(self,ori_probability=0.30):
        self.ori_probability = ori_probability
        #1:(0,0,0), 2:(90,0,0), 3:(180,0,0), 4:(270,0,0), 5:(0,90,0), 6:(0,270,0)
        self.face_to_you = [1,2,3,4,5,6]
        #rotate along z axis, so that we 24 combination totally
        # 1:0 degree, 2: 90 degree, 3: 180 degree, 4: 270 degree
        self.rotate_z = [1,2,3,4]

    def __call__(self, sample):
        if random.uniform(0,1) < self.ori_probability:
            return sample
        else:
            img, label, current_img = sample['image'], sample['label'], sample['img_idx']
            random_choise1=random.choice(self.face_to_you)
            rotated_img1 = {1: lambda x: x,
                            2: lambda x: ndimage.rotate(x,90,(1,2),reshape='True',mode = 'nearest'),
                            3: lambda x: ndimage.rotate(x,180,(1,2),reshape='True',mode = 'nearest'),
                            4: lambda x: ndimage.rotate(x,270,(1,2),reshape='True',mode = 'nearest'),
                            5: lambda x: ndimage.rotate(x,90,(0,2),reshape='True',mode = 'nearest'),
                            6: lambda x: ndimage.rotate(x,270,(0,2),reshape='True',mode = 'nearest')
                            }[random_choise1](img[0,...])
            random_choise2=random.choice(self.rotate_z)
            img[0,...] = {1: lambda x: x,
                            2: lambda x: ndimage.rotate(x,90,(0,1),reshape='True',mode = 'nearest'),
                            3: lambda x: ndimage.rotate(x,180,(0,1),reshape='True',mode = 'nearest'),
                            4: lambda x: ndimage.rotate(x,270,(0,1),reshape='True',mode = 'nearest')
                            }[random_choise2](rotated_img1)
                
        return {'image': img, 'label': label, 'img_idx': current_img}

class Flip(object):
    
    """
    Flip the image for data augmentation, but prefer original image.
    """
    
    def __init__(self,ori_probability=0.30):
        self.ori_probability = ori_probability

    def __call__(self, sample):
        if random.uniform(0,1) < self.ori_probability:
            return sample
        else:
            img, label, current_img = sample['image'], sample['label'], sample['img_idx']
            random_choise1=random.choice([1,2,3,4,5,6,7,8])
            img[0,...] = {1: lambda x: x,
                          2: lambda x: x[::-1,:,:],
                          3: lambda x: x[:,::-1,:],
                          4: lambda x: x[:,:,::-1],
                          5: lambda x: x[::-1,::-1,:],
                          6: lambda x: x[::-1,:,::-1],
                          7: lambda x: x[:,::-1,::-1],
                          8: lambda x: x[::-1,::-1,::-1]
                          }[random_choise1](img[0,...])
        return {'image': img, 'label': label, 'img_idx': current_img}

In [33]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class VGG_net(nn.Module):
    def __init__(self,conv_drop_rate=0.15,linear_drop_rate=0.4):
        super(VGG_net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=8, kernel_size=3,stride=1, padding=1)
        self.conv1_bn = nn.BatchNorm3d(8)
        self.conv2 = nn.Conv3d(in_channels=8, out_channels=8, kernel_size=3,stride=1,padding=1)
        self.conv2_bn = nn.BatchNorm3d(8)
        self.pool1 = nn.MaxPool3d(2, 2)
        self.dropout1 = nn.Dropout3d(conv_drop_rate)
        
        self.conv3 = nn.Conv3d(in_channels=8, out_channels=12, kernel_size=3,stride=1, padding=1)
        self.conv3_bn = nn.BatchNorm3d(12)
        self.conv4 = nn.Conv3d(in_channels=12, out_channels=12, kernel_size=3,stride=1, padding=1)
        self.conv4_bn = nn.BatchNorm3d(12)
        self.pool2 = nn.MaxPool3d(2, 2)
        self.dropout2 = nn.Dropout3d(conv_drop_rate)
        
        self.conv5 = nn.Conv3d(in_channels=12, out_channels=18, kernel_size=3,stride=1, padding=1)
        self.conv5_bn = nn.BatchNorm3d(18)
        self.conv6 = nn.Conv3d(in_channels=18, out_channels=18, kernel_size=3,stride=1, padding=1)
        self.conv6_bn = nn.BatchNorm3d(18)
        self.pool3 = nn.MaxPool3d(2, 2)
        self.dropout3 = nn.Dropout3d(conv_drop_rate)
        
        self.conv7 = nn.Conv3d(in_channels=18, out_channels=24, kernel_size=3,stride=1, padding=1)
        self.conv7_bn = nn.BatchNorm3d(24)
        self.conv8 = nn.Conv3d(in_channels=24, out_channels=24, kernel_size=3,stride=1, padding=1)
        self.conv8_bn = nn.BatchNorm3d(24)
        self.pool4 = nn.MaxPool3d(2, 2)
        self.dropout4 = nn.Dropout3d(conv_drop_rate)
        
        self.fc1 = nn.Linear(4*4*4*24, 128)
        self.fc1_bn = nn.BatchNorm1d(128)
        self.dropout5 = nn.Dropout(linear_drop_rate)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):        
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = self.dropout1(self.pool1(self.conv2_bn(F.relu(self.conv2(x)))))
        
        x = self.conv3_bn(F.relu(self.conv3(x)))
        x = self.dropout2(self.pool2(self.conv4_bn(F.relu(self.conv4(x)))))
        
        x = self.conv5_bn(F.relu(self.conv5(x)))
        x = self.dropout3(self.pool3(self.conv6_bn(F.relu(self.conv6(x)))))
        
        x = self.conv7_bn(F.relu(self.conv7(x)))
        x = self.dropout4(self.pool4(self.conv8_bn(F.relu(self.conv8(x)))))
        
        x = x.view(-1, 4*4*4*24)
        x = self.dropout5(self.fc1_bn(F.relu(self.fc1(x))))
        x = self.fc2(x)
        return x


In [34]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['label']  
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i_batch % 10 == 0:
            print("epoch {}, batch {}, current loss {}".format(epoch+1,i_batch,running_loss/10))
            running_loss = 0.0


In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

num_epochs = 3
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.5,1.0]).to(device))
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
print('choose SGD as optimizer')
#optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
#print('choose Adam as optimizer')

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
for epoch in range(num_epochs):
    scheduler.step()
    idx1=np.random.choice(range(len(train_idx)),89600,replace=False)
    train_data_shuffle =[train_idx[idx1[i]] for i in range(len(idx1))]
    Mouse_dataset = Mouse_sub_volumes(all_train_data,train_data_shuffle,
                                     transform=transforms.Compose([Rotate(),Flip()]))
    dataloader = DataLoader(Mouse_dataset, batch_size=128,
                        shuffle=True, num_workers=4)
    train(net, device, dataloader, optimizer, criterion, epoch)

Let's use 2 GPUs!
There are 247842 parameters in the model
choose SGD as optimizer
epoch 1, batch 0, current loss 0.07530946135520936
epoch 1, batch 10, current loss 0.7062430560588837
epoch 1, batch 20, current loss 0.652978903055191
epoch 1, batch 30, current loss 0.627645218372345
epoch 1, batch 40, current loss 0.5988895893096924
epoch 1, batch 50, current loss 0.5971351623535156
epoch 1, batch 60, current loss 0.5916464686393738
epoch 1, batch 70, current loss 0.5681986391544342
epoch 1, batch 80, current loss 0.5575403988361358
epoch 1, batch 90, current loss 0.5426340520381927
epoch 1, batch 100, current loss 0.5271922469139099
epoch 1, batch 110, current loss 0.5266867458820343
epoch 1, batch 120, current loss 0.5251856058835983
epoch 1, batch 130, current loss 0.4791441559791565
epoch 1, batch 140, current loss 0.4685778945684433
epoch 1, batch 150, current loss 0.49620257019996644
epoch 1, batch 160, current loss 0.48142257928848264
epoch 1, batch 170, current loss 0.45542372

epoch 3, batch 140, current loss 0.009507321508135647
epoch 3, batch 150, current loss 0.016762164968531577
epoch 3, batch 160, current loss 0.010308969882316887
epoch 3, batch 170, current loss 0.017509105033241212
epoch 3, batch 180, current loss 0.012462080549448728
epoch 3, batch 190, current loss 0.009122524556005374
epoch 3, batch 200, current loss 0.016121569101233035
epoch 3, batch 210, current loss 0.0064844349748454985
epoch 3, batch 220, current loss 0.014309449295978993
epoch 3, batch 230, current loss 0.006916796858422458
epoch 3, batch 240, current loss 0.00963301642332226
epoch 3, batch 250, current loss 0.007315920165274293
epoch 3, batch 260, current loss 0.012476668390445412
epoch 3, batch 270, current loss 0.009849526756443083
epoch 3, batch 280, current loss 0.008650027180556208
epoch 3, batch 290, current loss 0.010360892536118627
epoch 3, batch 300, current loss 0.01320838239043951
epoch 3, batch 310, current loss 0.007286943926010281
epoch 3, batch 320, current l

In [36]:
print(len(train_idx))
print(len(test_idx))

236874
42962


In [37]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# net = VGG_net()
# #net.apply(weight_init)
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     net = nn.DataParallel(net)
# net.to(device)
# print("There are {} parameters in the model".format(count_parameters(net)))

In [38]:
def test(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    

def test2(model, device, test_loader, test_img_list):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    test_dic = {}
    for idx in test_img_list:
        test_dic[idx] = [0,0]
        
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels, current_img = sample_batched['image'], sample_batched['label'], sample_batched['img_idx']    
            inputs = inputs.to(device)
            current_img = current_img.numpy()
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            for i in range(len(labels)):
                test_dic[current_img[i]][predicted[i].cpu().numpy()] +=1
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return test_dic

In [39]:
# torch.save(net.state_dict(), './model/mut_clas_e1_v0105_2.pth')

In [40]:
# net.load_state_dict(torch.load('./model/mut_clas_e1_v1107.pth'))

In [41]:
# print('train accuracy: ')
# Mouse_dataset = Mouse_sub_volumes(all_train_data,train_idx)
# train_dataloader = DataLoader(Mouse_dataset, batch_size=128,
#                         shuffle=False, num_workers=4)
# test(net, device, train_dataloader)

In [42]:
print('test accuracy: ')
Mouse_dataset = Mouse_sub_volumes(all_train_data,test_idx)
test_dataloader = DataLoader(Mouse_dataset, batch_size=128,
                        shuffle=False, num_workers=4)
test_dic = test2(net, device, test_dataloader, test_img_list)

test accuracy: 
total_num:42962, test accuracy:0.8501466412178204, positive_acc:0.9205470257571944, negative_acc:0.7623928048525414


In [43]:
import copy
test_dic2 = copy.deepcopy(test_dic)
for key in test_dic:
    if test_dic[key][0] > test_dic[key][1]:
        test_dic2[key][0] = 'pred_as_mul'
    else:
        test_dic2[key][0] = 'pred_as_nor'
    if key in test_mutant_list:
        test_dic2[key][1] = 'true_as_mul'
    elif key in test_normal_list:
        test_dic2[key][1] = 'true_as_nor'
print(test_dic2)

{255: ['pred_as_mul', 'true_as_mul'], 256: ['pred_as_mul', 'true_as_mul'], 257: ['pred_as_mul', 'true_as_mul'], 258: ['pred_as_nor', 'true_as_mul'], 268: ['pred_as_mul', 'true_as_mul'], 290: ['pred_as_mul', 'true_as_mul'], 293: ['pred_as_mul', 'true_as_mul'], 294: ['pred_as_mul', 'true_as_mul'], 300: ['pred_as_mul', 'true_as_mul'], 304: ['pred_as_nor', 'true_as_mul'], 305: ['pred_as_nor', 'true_as_mul'], 306: ['pred_as_mul', 'true_as_mul'], 307: ['pred_as_nor', 'true_as_mul'], 312: ['pred_as_mul', 'true_as_mul'], 316: ['pred_as_mul', 'true_as_mul'], 259: ['pred_as_nor', 'true_as_nor'], 260: ['pred_as_nor', 'true_as_nor'], 261: ['pred_as_nor', 'true_as_nor'], 262: ['pred_as_nor', 'true_as_nor'], 263: ['pred_as_nor', 'true_as_nor'], 264: ['pred_as_nor', 'true_as_nor'], 265: ['pred_as_nor', 'true_as_nor'], 266: ['pred_as_nor', 'true_as_nor'], 267: ['pred_as_nor', 'true_as_nor'], 269: ['pred_as_nor', 'true_as_nor'], 270: ['pred_as_nor', 'true_as_nor'], 271: ['pred_as_nor', 'true_as_nor'], 

In [44]:
test_dic3 = {}
for key in test_dic:
        test_dic3[key] = test_dic[key][0]/(test_dic[key][0]+test_dic[key][1])
print(test_dic3)

{255: 1.0, 256: 1.0, 257: 1.0, 258: 0.0, 268: 0.9921197793538219, 290: 0.5047619047619047, 293: 1.0, 294: 1.0, 300: 1.0, 304: 0.01542111506524318, 305: 0.0042780748663101605, 306: 0.9451058201058201, 307: 0.23380281690140844, 312: 0.8044382801664355, 316: 0.9682966147232671, 259: 0.0, 260: 0.0, 261: 0.0, 262: 0.0, 263: 0.0, 264: 0.0, 265: 0.0, 266: 0.0, 267: 0.0, 269: 0.0, 270: 0.0, 271: 0.0, 272: 0.0, 273: 0.0, 274: 0.0, 275: 0.029166666666666667, 276: 0.0, 277: 0.0, 278: 0.050347222222222224, 279: 0.0, 280: 0.0, 281: 0.0, 282: 0.0, 283: 0.0, 284: 0.0, 285: 0.0, 286: 0.04377104377104377, 287: 0.9888888888888889, 288: 0.0, 289: 0.0, 291: 0.03676470588235294, 292: 0.004273504273504274, 295: 0.041666666666666664, 296: 1.0, 297: 0.003787878787878788, 298: 0.0, 299: 0.0, 301: 0.24206349206349206, 302: 0.0, 303: 0.0, 308: 0.0, 309: 0.0, 310: 0.0, 311: 0.007407407407407408, 313: 0.0, 314: 0.4722222222222222, 315: 0.35785953177257523, 317: 0.0, 318: 0.3619047619047619, 319: 0.0, 320: 0.0, 321

In [45]:
cross_table = np.zeros([2,2])
mut_to_nor = []
nor_to_mul = []
thre = 0.5
for key in test_dic:
    if test_dic3[key] >= thre and key in test_mutant_list:
        cross_table[0,0] += 1
    elif  test_dic3[key] < thre and key in test_mutant_list:
        cross_table[0,1] += 1
        mut_to_nor.append(key)
    elif test_dic3[key] >= thre and key in test_normal_list:
        cross_table[1,0] += 1
        nor_to_mul.append(key)
    elif test_dic3[key] < thre and key in test_normal_list:
        cross_table[1,1] += 1
print(cross_table)
print(mut_to_nor)
print(nor_to_mul)

[[ 11.   4.]
 [  5.  79.]]
[258, 304, 305, 307]
[287, 296, 338, 354, 369]


In [ ]:
for i in mut_to_nor:
    print(i, ': ', test_dic[i])

In [ ]:
for i in nor_to_mul:
    print(i, ': ', test_dic[i])

In [ ]:
for i in mut_to_nor:
    print(i)
    img_nft = nib.Nifti1Image(np.squeeze(all_train_data[i]),np.eye(4))
    img_save_data_path = './img/mul_img{}.nii'.format(i)
    nib.save(img_nft,img_save_data_path)

In [ ]:
for i in nor_to_mul:
    print(i)
    img_nft = nib.Nifti1Image(np.squeeze(all_train_data[i]),np.eye(4))
    img_save_data_path = './img/mul_img{}.nii'.format(i)
    nib.save(img_nft,img_save_data_path)

In [ ]:
def test2(model, device, test_loader, test_img_list):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    test_dic = {}
    for idx in test_img_list:
        test_dic[idx] = [0,0]
        
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels, current_img = sample_batched['image'], sample_batched['label'], sample_batched['img_idx']    
            inputs = inputs.to(device)
            current_img = current_img.numpy()
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            for i in range(len(labels)):
                test_dic[current_img[i]][predicted[i].cpu().numpy()] +=1
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return test_dic

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(cross_table, index=['multant','normal'],columns=['pred as mul','pred as nor'])
print(df)